In [ ]:
!pip install weaviate-client langchain tiktoken pypdf rapidocr-onnxruntime

In [ ]:
WEAVIATE_CLUSTER="https://mylangchainproject-z88ava1x.weaviate.network"
WEAVIATE_API_KEY="" # Add your Weaviate API key here

In [ ]:
from langchain.vectorstores import Weaviate
import weaviate

WEAVIATE_URL = WEAVIATE_CLUSTER
WEAVIATE_API_KEY = WEAVIATE_API_KEY

client = weaviate.Client(
    url=WEAVIATE_URL, auth_client_secret=weaviate.AuthApiKey(WEAVIATE_API_KEY)
)

In [ ]:
# fixing unicode error in google colab
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install sentence-transformers

In [ ]:
# specify embedding model (using huggingface sentence transformer)
from langchain.embeddings import HuggingFaceEmbeddings
embedding_model_name = "sentence-transformers/all-mpnet-base-v2"
#model_kwargs = {"device": "cuda"}
embeddings = HuggingFaceEmbeddings(
  model_name=embedding_model_name,
  #model_kwargs=model_kwargs
)

# you can load multiple types of pdf using the langchain just check with the document

https://python.langchain.com/docs/modules/data_connection/document_loaders/pdf/

In [ ]:
from langchain.document_loaders import PyPDFLoader
loader = PyPDFLoader("/content/RAG.pdf", extract_images=True)
pages = loader.load()

In [ ]:
pages

In [ ]:
# Split text into chunks
from langchain.text_splitter import RecursiveCharacterTextSplitter
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=20)
docs = text_splitter.split_documents(pages)

In [ ]:
docs

In [ ]:
vector_db = Weaviate.from_documents(
    docs, embeddings, client=client, by_text=False
)

In [ ]:
print(vector_db.similarity_search("what is rag?", k=3)[0].page_content)

In [ ]:
print(vector_db.similarity_search("what is rag?", k=3)[1].page_content)

In [ ]:
print(vector_db.similarity_search("what is rag?", k=3)[2].page_content)

In [ ]:
print(
    vector_db.similarity_search(
        "what is attention?", k=3)
    )

In [ ]:
from langchain.prompts import ChatPromptTemplate

template="""You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use ten sentences maximum and keep the answer concise.
Question: {question}
Context: {context}
Answer:
"""

In [ ]:
prompt=ChatPromptTemplate.from_template(template)

In [ ]:
prompt

In [ ]:
from langchain import HuggingFaceHub

In [ ]:
from google.colab import userdata
huggingfacehub_api_token=userdata.get('HuGGINGFACE_TOKEN')

In [ ]:
model = HuggingFaceHub(
    huggingfacehub_api_token=huggingfacehub_api_token,
    repo_id="mistralai/Mistral-7B-Instruct-v0.1",
    model_kwargs={"temperature":1, "max_length":180}
)

In [ ]:
from langchain.schema.runnable import RunnablePassthrough
from langchain.schema.output_parser import StrOutputParser

In [ ]:
output_parser=StrOutputParser()

In [ ]:
retriever=vector_db.as_retriever()

In [ ]:
rag_chain = (
    {"context": retriever,  "question": RunnablePassthrough()}
    | prompt
    | model
    | output_parser
)

In [ ]:
print(rag_chain.invoke("what is rag system?"))

In [ ]:
print(rag_chain.invoke("How does the RAG model differ from traditional language generation models?"))